In [1]:
import requests
from pprint import pprint
import re
import lxml.html
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def log_error(line1, line2):
    _line1 = str(line1)
    _line2 = str(line2)
    diff = 0
    chars1 = len(_line1) + 10
    chars2 = len(_line2) + 10
    if chars1 > chars2:
        diff = chars1 - chars2
        if diff % 2 != 0:
            diff = int(diff - .5)
            _line2 = '# ' + _line2
        else:
            _line2 = ' ' + _line2
        main_hashes = ''.join(['#' for i in range(chars1)])
        hashes = ''.join(['#' for i in range(int(diff/2)+4)])
        log_line = (f"{main_hashes}\n"
                    f"#### {_line1} ####\n"
                    f"{hashes}{_line2} {hashes}\n"                    
                    f"{main_hashes}\n")
        return(log_line)
   
    elif chars2 > chars1:
        diff = chars2 - chars1
        if diff % 2 != 0:
            diff = int(diff - .5)
            _line1 = '# ' + _line1
        else:
            _line1 = ' ' + _line1
        main_hashes = ''.join(['#' for i in range(chars2)])
        hashes = ''.join(['#' for i in range(int(diff/2)+4)])
        log_line = (f"{main_hashes}\n"
                    
                    f"{hashes}{_line1} {hashes}\n"
                    f"#### {_line2} ####\n"
                    f"{main_hashes}\n")
        return(log_line)
        
    else:
        hashes = '####'
        main_hashes = ''.join(['#' for i in range(chars2)])
        log_line = (f"{main_hashes}\n"
                    f"#### {_line1} ####\n"
                    f"{hashes} {_line2} {hashes}\n"
                    f"{main_hashes}\n")
        return(log_line)
        

In [3]:
# init session 
S = requests.Session()

# set URL
URL = "https://es.wikipedia.org/w/api.php"

In [4]:
page_id = "700615"
topic = 'city_region'
subtopic = 'spain'

In [5]:
# get sections
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "sections"
}

sectionsR = S.get(url=URL, params=PARAMS)
sectionDATA = sectionsR.json()



In [6]:
sectionDATA

{'parse': {'title': 'Anexo:Municipios de España por población',
  'pageid': 700615,
  'sections': [{'toclevel': 1,
    'level': '2',
    'line': 'Municipios con más de 100&nbsp;000 habitantes',
    'number': '1',
    'index': '1',
    'fromtitle': 'Anexo:Municipios_de_España_por_población',
    'byteoffset': 875,
    'anchor': 'Municipios_con_más_de_100_000_habitantes'},
   {'toclevel': 1,
    'level': '2',
    'line': 'Municipios de más de 50&nbsp;000 y menos de 100&nbsp;000 habitantes',
    'number': '2',
    'index': '2',
    'fromtitle': 'Anexo:Municipios_de_España_por_población',
    'byteoffset': 7599,
    'anchor': 'Municipios_de_más_de_50_000_y_menos_de_100_000_habitantes'},
   {'toclevel': 1,
    'level': '2',
    'line': 'Municipios de más de 20&nbsp;000 y menos de 50&nbsp;000 habitantes',
    'number': '3',
    'index': '3',
    'fromtitle': 'Anexo:Municipios_de_España_por_población',
    'byteoffset': 16257,
    'anchor': 'Municipios_de_más_de_20_000_y_menos_de_50_000_habit

In [ ]:
sections = [sec['anchor'] for sec in sectionDATA["parse"]["sections"]]

In [7]:
# create section list filtering out non-data sections
filt = [
    'References',
    'See_also',
    'External_links',
    #"Historical_names_of_Valletta's_streets_and_squares"
]

incl = [
    'Municipios_con_más_de_100_000_habitantes',
    'Municipios_de_más_de_50_000_y_menos_de_100_000_habitantes',
    'Municipios_de_más_de_20_000_y_menos_de_50_000_habitantes'
    
]

# sec_list = [sec for sec in sectionDATA["parse"]["sections"] if not sec['anchor'] in filt]
sec_list = [sec for sec in sectionDATA["parse"]["sections"] if sec['anchor'] in incl]


len(sec_list)
sec_list

[{'toclevel': 1,
  'level': '2',
  'line': 'Municipios con más de 100&nbsp;000 habitantes',
  'number': '1',
  'index': '1',
  'fromtitle': 'Anexo:Municipios_de_España_por_población',
  'byteoffset': 875,
  'anchor': 'Municipios_con_más_de_100_000_habitantes'},
 {'toclevel': 1,
  'level': '2',
  'line': 'Municipios de más de 50&nbsp;000 y menos de 100&nbsp;000 habitantes',
  'number': '2',
  'index': '2',
  'fromtitle': 'Anexo:Municipios_de_España_por_población',
  'byteoffset': 7599,
  'anchor': 'Municipios_de_más_de_50_000_y_menos_de_100_000_habitantes'},
 {'toclevel': 1,
  'level': '2',
  'line': 'Municipios de más de 20&nbsp;000 y menos de 50&nbsp;000 habitantes',
  'number': '3',
  'index': '3',
  'fromtitle': 'Anexo:Municipios_de_España_por_población',
  'byteoffset': 16257,
  'anchor': 'Municipios_de_más_de_20_000_y_menos_de_50_000_habitantes'}]

In [ ]:
for index, line in enumerate(lines):
    print(index, line)

In [132]:
# create nested section_dict dictionary from section list based on toclevel
# refactored
section_dict = {}
current_parent = None
for i in range(len(sec_list)):
    if i == 0 and len(sec_list) > 1:
        if sec_list[i]['toclevel'] < sec_list[i + 1]['toclevel']:
            section_dict[sec_list[i]['line']] = {}
            current_grandparent = sec_list[i]['line']
        else:
            section_dict[sec_list[i]['line']] = []

    elif i != len(sec_list) - 1:
        if sec_list[i - 1]['toclevel'] < sec_list[i]['toclevel'] and sec_list[i]['toclevel'] < sec_list[i + 1]['toclevel']:
            section_dict[current_grandparent][sec_list[i]['line']] = {}
            current_parent = section_dict[current_grandparent][sec_list[i]['line']]
            
        elif sec_list[i]['toclevel'] == sec_list[i + 1]['toclevel'] or sec_list[i]['toclevel'] > sec_list[i + 1]['toclevel']:
            if current_parent != None:
                current_parent[sec_list[i]['line']] = []
            else: section_dict[sec_list[i]['line']] = []
            
        elif sec_list[i]['toclevel'] < sec_list[i + 1]['toclevel']:
            section_dict[current_grandparent][sec_list[i]['line']] = {}
            current_parent = section_dict[current_grandparent][sec_list[i]['line']]
            
    else:
        if sec_list[i]['toclevel'] == sec_list[i - 1]['toclevel']:
            section_dict[sec_list[i]['line']] = []
        else:
            section_dict[sec_list[i]['line']] = []
              

#pprint(section_dict)

In [133]:
keys = [key for key in section_dict.keys()]
for key in keys:
    print(key)
    
    section_dict[re.sub(r'\&nbsp;', ',', key)] = section_dict.pop(key)
    
section_dict

Municipios con más de 100&nbsp;000 habitantes
Municipios de más de 50&nbsp;000 y menos de 100&nbsp;000 habitantes
Municipios de más de 20&nbsp;000 y menos de 50&nbsp;000 habitantes


{'Municipios con más de 100,000 habitantes': [],
 'Municipios de más de 50,000 y menos de 100,000 habitantes': [],
 'Municipios de más de 20,000 y menos de 50,000 habitantes': []}

In [52]:
# hard coded name for section in section dict bec inconsistent wikipedia format
section_dict = {'San Jose': []}

In [ ]:
# create nested section_dict dictionary from section list based on toclevel
# adding 'Z' section by hand consider refactor
section_dict = {}
is_new_parent = True
parent = ''
for i in range(len(sec_list) - 1):
    if sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 1:
        section_dict[sec_list[i]['anchor']] = []
    elif sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 2:
        section_dict[sec_list[i]['anchor']] = {}
        parent = sec_list[i]['anchor']
    elif sec_list[i]['toclevel'] == 2:
        section_dict[parent][sec_list[i]['anchor']] = []
section_dict['Z'] = []
#section_dict

In [ ]:
# make list of top level sections
secs = [sec for sec, val in section_dict.items()]


In [ ]:
# make section dict if no nested sections
section_dict = {sec['anchor']:[] for sec in sec_list}

In [ ]:
# get links
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "links"
}

linkR = S.get(url=URL, params=PARAMS)
linkDATA = linkR.json()

pprint(linkDATA["parse"]["links"])

In [ ]:
links = [link['*'] for link in linkDATA["parse"]["links"]]
for i, link in enumerate(links):
    print(i, link)

In [ ]:
# visualize link data
for link in links[3:4]:
    # get page info
    PARAMS = {
        "action": "query",
        "titles": link,
        "format": "json",
        "prop": "info",
        'inprop': "displaytitle"
    }

    infoR = S.get(url=URL, params=PARAMS)
    infoDATA = infoR.json()
    pprint(infoDATA)

In [11]:
# get data
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "wikitext"
}

textR = S.get(url=URL, params=PARAMS)
textDATA = textR.json()


In [12]:
# parse data

lines = textDATA["parse"]["wikitext"]["*"].split('\n')

In [13]:
for index, line in enumerate(lines[:2414]):
    print(index, line)

0 {{PA|referencias|desactualizado}}
1 [[Archivo:Municipalities of Spain.svg|thumb|450px|right|Mapa municipal de España]]
2 En 2019, en [[España]] existen 8131 [[Municipio (España)|municipios]], repartidos por las cincuenta [[Provincias de España|provincias]], [[Ceuta]] y [[Melilla]] y cuyos datos oficiales de población son recogidos por el [[Instituto Nacional de Estadística (España)|Instituto Nacional de Estadística.]]<ref>[http://www.ine.es/daco/daco42/codmun/codmunmapa.htm Relación de municipios y códigos por provincias]</ref> En las siguientes tablas se recogen los '''municipios españoles ordenados por población''' según los datos oficiales del [[Instituto Nacional de Estadística (España)|Instituto Nacional de Estadística]]. Las capitales de [[provincia]] están marcadas en '''negrita''' y las capitales de [[comunidad autónoma]] están en {{Versalitas|versalitas}}.
3 
4 == Municipios con más de 100{{esd}}000 habitantes ==
5 
6 <center>
7 {| class="wikitable sortable"
8 |- style="back

886 | 144
887 |[[Mollet del Vallés]]
888 |align=right| 51&nbsp;133
889 | [[Provincia de Barcelona|Barcelona]]
890 | {{ESP|CT}}
891 |-
892 | 145
893 |[[Villarreal]]
894 |align=right| 50&nbsp;577
895 | [[Provincia de Castellón|Castellón]]
896 | {{ESP|VC}}
897 |-
898 |}
899 </center>
900 
901 == Municipios de más de 20{{esd}}000 y menos de 50{{esd}}000 habitantes ==
902 <center>
903 {| class="wikitable sortable"
904 |- style="background:#e3e3e3"
905 ! #
906 ! Nombre
907 ! Población 
908 (2016)
909 ! Provincia
910 ! Comunidad autónoma
911 |-
912 | 146
913 |[[Ibiza (ciudad)|Ibiza]]
914 | align=right| 49 549
915 | [[Islas Baleares]]
916 | {{ESP|IB}}
917 |-
918 | 147
919 |[[Puertollano]]
920 |align=right| 49 166
921 | [[Provincia de Ciudad Real|Ciudad Real]]
922 | {{ESP|CM}}
923 |-
924 | 148
925 | [[Calviá]]
926 |align=right| 49 063
927 | [[Islas Baleares]]
928 | {{ESP|IB}}
929 |-
930 | 149
931 |[[Colmenar Viejo]]
932 |align=right| 48 020
933 | [[Comunidad de Madrid|Madrid]]
934 | {{ESP|MD}}


1762 | {{ESP|CN}}
1763 |-
1764 | 288
1765 | [[Illescas (Toledo)|Illescas]]
1766 |align=right| 26 672
1767 | [[Provincia de Toledo|Toledo]]
1768 | {{ESP|CM}}
1769 |-
1770 | 289
1771 | [[Cangas de Morrazo]]
1772 |align=right| 26 584
1773 | [[Provincia de Pontevedra|Pontevedra]]
1774 | {{ESP|GA}}
1775 |-
1776 | 290
1777 | [[Benicarló]]
1778 |align=right| 26 486
1779 | [[Provincia de Castellón|Castellón]]
1780 | {{ESP|VC}}
1781 |-
1782 | 291
1783 | [[Salou]]
1784 |align=right| 26 386
1785 | [[Provincia de Tarragona|Tarragona]]
1786 | {{ESP|CT}}
1787 |-
1788 | 292
1789 | [[Pineda de Mar]]
1790 |align=right| 26 240
1791 | [[Provincia de Barcelona|Barcelona]]
1792 | {{ESP|CT}}
1793 |-
1794 | 293
1795 | [[Novelda]]
1796 |align=right| 26 054
1797 | [[Provincia de Alicante|Alicante]]
1798 | {{ESP|VC}}
1799 |-
1800 | 294
1801 | [[Villanueva de la Serena]]
1802 |align=right| 25 992
1803 | [[Provincia de Badajoz|Badajoz]]
1804 | {{ESP|EX}}
1805 |-
1806 | 295
1807 | [[San José (Baleares)|San José]]


In [84]:
this_sec_name

'Cities'

In [147]:
# parse 
#       WikiText
# data


debug = False
special = True

this_sec_name = None
this_sec = None

this_item = {}

new_cols = False
cols = []
item_counter = 0

for index, line in enumerate(lines[:2414]):
    #print(line)
    #if index > 50 and index < 57:
    #    debug = True
    #else:
    #    continue
    if debug:
        if index != 0: 
            print('$\n' + str(index -1) + '\n' + lines[index - 1] + '\n' + str(index) + '\n' + line + '\n$')
            print(index, re.match('\|\-', lines[index - 1][0:2]))
            print(line)
            print(index,re.match('\!\w+', line[0:2]))
            this_sec = []
            cols = ['#', 'Nombre', 'Población', 'Comunidad']

    if re.match('==\s\w', line[0:4]):
        if debug or special: print("$$-2-$$")
        this_sec_name = line.strip().replace("=", "").replace("{{esd}}", ",").strip()
        print(this_sec_name)

    if re.match('\{\|', lines[index - 2][0:2]) and re.match('\!\s\w+|\!\s#', line[0:3]):
        if debug or special: print("$-6-$$")
        cols = []
        new_cols = True

    if re.match('\!\s\w+', lines[index - 1][0:3]) and re.match('\|\-', line[0:2]):
        if debug: print("$-7-$$")
        new_cols = False
        this_sec = []
        
        #this_sec_start = index + 2

    if new_cols == True:
        if debug: print("$-8-$$")
        #print(line)
        #print(index)
        if re.match(r'\!\s', line[0:2]):
            cols.append(re.search(r'\!\s(\w+|#)', line).group(1))
        #print(cols)

    if this_sec != None:
        if debug: print("$-10-$$")
        #print(line)
        
        
        # new item dict
        
        if re.match('\|\-', lines[index][0:2]) and not debug:
            this_item = {col: '' for col in cols}
            item_counter = 0
            #print(item_counter)
            #print(index)
            if debug: print("$-11-$$")
            #for i in range(len(cols)):
        
        
        # rank
        
        if re.match('\|\s*\d', lines[index][0:4]):
            if debug:
                print("$$-11-$$")
                print(index)
                print(line)
            this_item[cols[0]] = re.search(r'\d+', line).group(0)
            #print(this_item[cols[0]])
            item_counter += 1
        
        
        # city name
        
        if re.match("\|'", lines[index][0:2]):   
            #print(line)
            #print(index)
            if "{" in line:
                this_item[cols[1]] = re.search(r"\|'''\{\{\w*\|\[\[(.*)\]\]\}\}'''", line).group(1)
                
            else:
                #print(index)
                #print(line)
                if "|" in line[1:]:
                    this_item[cols[1]] = re.search(r"\|'''\[\[(.*)\|(.*)\]\]'''", line).group(2)
                else:
                    this_item[cols[1]] = re.search(r"\|'''\[\[(.*)\]\]'''", line).group(1)
            item_counter += 1
         
        if re.match("\|\s*\[", lines[index][0:3]) and item_counter < 3:        
            #print(index)
            #print(line)
            if "|" in line[1:]:
                #print(index)
                #print(line)
                this_item[cols[1]] = re.search(r"\|\s*\[\[(.*)\|(.*)\]\]", line).group(2)
                
            else:
                this_item[cols[1]] = re.search(r"\|\s*\[\[(.*)\]\]", line).group(1)
            item_counter += 1

        if re.match("\|\{", lines[index][0:2]) and item_counter < 3:        
            #print(index)
            #print(line)
            if "|" in line[14:]:
                #print(index)
                #print(line)
                this_item[cols[1]] = re.search(r"\|\{\{\w*\|\[\[(.*)\|(.*)\]\]\}\}", line).group(2)
            else:
                this_item[cols[1]] = re.search(r"\|\{\{\w*\|\[\[(.*)\]\]\}\}", line).group(1)
            item_counter += 1
        
        
        # population
        
        if re.match('\|\s*a', lines[index][0:3]):
            this_line = re.search(r'\|.*\|\s*(.*)', line).group(1)
            if '&nbsp' in this_line:
                #print('@@#000#@@')
                #print(index)
                #print(line)
                this_item[cols[2]] = re.sub(r'\&nbsp;', ',', this_line)
                this_item[cols[2]] = re.sub(r',', '', this_item[cols[2]])
                this_item[cols[2]] = int(re.sub(r'\s', '', this_item[cols[2]]))
                
            else:
                #print('@@#111#@@')
                #print(index)
                #print(this_line)
                this_item[cols[2]] = re.sub(r'\s', ',', this_line)
                this_item[cols[2]] = re.sub(r',', '', this_item[cols[2]])
                this_item[cols[2]] = int(re.sub(r'\s', '', this_item[cols[2]]))
            item_counter += 1
        
        
        # province
        
        if re.match('\|\s*\[', lines[index][0:3]) and item_counter > 2:
            #print(line)
            #print(index)
            if "|" in line[1:]:
                #print("pipe")
                #print(index)
                #print(line)
                this_item[cols[3]] = re.search(r'\|\s*\[\[(\w*\s*\W*)*\|(.*)\]\]', line).group(2)
                
            else:
                #print("$NO$pipe")
                #print(index)
                #print(line)
                this_item[cols[3]] = re.search(r'\[\[(.*)\]\]', line).group(1)
            item_counter += 1
        
        
        # autonomous community
        
        if re.match('\|\s\{', lines[index][0:3]):    
            this_item[cols[4]] = line
            item_counter += 1
            if debug == False:
                if this_item['#'] == '':
                    print(this_item)
                this_sec.append(this_item)
                section_dict[this_sec_name] = this_sec
            #print(this_item)

    # if this_item != None:

              
                   
                    
                    
    

$$-2-$$
Municipios con más de 100,000 habitantes
$-6-$$
$$-2-$$
Municipios de más de 50,000 y menos de 100,000 habitantes
$-6-$$
$$-2-$$
Municipios de más de 20,000 y menos de 50,000 habitantes
$-6-$$


In [95]:
cols = ['#', 'Nombre', 'Población', 'Comunidad']

['#', 'Nombre', 'Población']

In [98]:
section_dict

{'Municipios con más de 100,000 habitantes': [{'#': '1',
   'Nombre': 'Madrid',
   'Población': '3,266 126',
   'Provincia': 'Madrid',
   'Comunidad': '| {{ESP|MD}}'},
  {'#': '2',
   'Nombre': 'Barcelona',
   'Población': '1,636 762',
   'Provincia': 'Barcelona',
   'Comunidad': '| {{ESP|CT}}'},
  {'#': '3',
   'Nombre': 'Valencia',
   'Población': '794,288',
   'Provincia': 'Valencia',
   'Comunidad': '| {{ESP|VC}}'},
  {'#': '4',
   'Nombre': 'Sevilla',
   'Población': '688,592',
   'Provincia': 'Sevilla',
   'Comunidad': '| {{ESP|AN}}'},
  {'#': '5',
   'Nombre': 'Zaragoza',
   'Población': '674,997',
   'Provincia': 'Zaragoza',
   'Comunidad': '| {{ESP|AR}}'},
  {'#': '6',
   'Nombre': 'Málaga',
   'Población': '574,654',
   'Provincia': 'Málaga',
   'Comunidad': '| {{ESP|AN}}'},
  {'#': '7',
   'Nombre': 'Murcia',
   'Población': '453,258',
   'Provincia': 'Murcia',
   'Comunidad': '| {{ESP|MC}}'},
  {'#': '8',
   'Nombre': 'Palma de Mallorca',
   'Población': '416,065',
   'Prov

In [138]:
for section, cities in section_dict.items():
    for city in cities:
        for stat, value in city.items():
            if value == '':
                print(city)
        

In [151]:
cities = []
for section, citys in section_dict.items():
    for city in citys:
        cities.append(city)


In [152]:
# items in cities table
total = sum([city['Población']for city in cities])
cities = [city['Nombre'] + ', ' + city['Provincia']  + ', ' + str(round(city['Población']/total, 5)) for city in cities]
cities

['Madrid, Madrid, 0.1024',
 'Barcelona, Barcelona, 0.05131',
 'Valencia, Valencia, 0.0249',
 'Sevilla, Sevilla, 0.02159',
 'Zaragoza, Zaragoza, 0.02116',
 'Málaga, Málaga, 0.01802',
 'Murcia, Murcia, 0.01421',
 'Palma de Mallorca, Islas Baleares, 0.01304',
 'Las Palmas de Gran Canaria, Las Palmas, 0.01191',
 'Bilbao, Vizcaya, 0.01087',
 'Alicante, Alicante, 0.0105',
 'Córdoba, Córdoba, 0.01021',
 'Valladolid, Valladolid, 0.00936',
 'Vigo, Pontevedra, 0.00926',
 'Gijón, Asturias, 0.00852',
 'Hospitalet de Llobregat, Barcelona, 0.00831',
 'Vitoria, Álava, 0.00789',
 'La Coruña, La Coruña, 0.0077',
 'Elche, Alicante, 0.00729',
 'Granada, Granada, 0.00729',
 'Tarrasa, Barcelona, 0.00691',
 'Badalona, Barcelona, 0.00691',
 'Oviedo, Asturias, 0.00689',
 'Cartagena, Murcia, 0.00673',
 'Sabadell, Barcelona, 0.0067',
 'Jerez de la Frontera, Cádiz, 0.00667',
 'Móstoles, Madrid, 0.00656',
 'Santa Cruz de Tenerife, Santa Cruz de Tenerife, 0.0065',
 'Pamplona, Navarra, 0.00632',
 'Almería, Almería,

In [33]:
# make 1 list
all_items = [item for section, items in section_dict.items() for item in items]
# make 2 lists, nested
# all_items = [item for section, item in section_dict.items()]
# flatten lists 2 ways
# all_items = [lyst1 for lyst2 in all_items for lyst1 in lyst2]
#all_items = [item for items in all_items for item in items]
all_items 

['19th Avenue (San Francisco)|19th Avenue',
 'California Street (San Francisco)|California Street',
 'Fell Street',
 'Geary Boulevard',
 'Grant Avenue',
 'Fulton Street(San Francisco)|Fulton Street',
 'Lincoln Way (San Francisco)|Lincoln Way',
 'Lombard Street (San Francisco)|Lombard Street',
 'Market Street (San Francisco)|Market Street',
 'Park Presidio Boulevard',
 'Portola Drive',
 'Van Ness Avenue (San Francisco)|Van Ness Avenue',
 '24th Street (San Francisco)|24th Street',
 'Columbus Avenue (San Francisco)|Columbus Avenue',
 'Fillmore Street',
 'Kearny Street',
 'Mission Street',
 'Polk Street',
 'Stockton Street (San Francisco)|Stockton Street',
 'Union Street (San Francisco)|Union Street',
 'Third Street (San Francisco)|Third Street',
 '22nd Street (San Francisco)',
 '49-Mile Scenic Drive',
 'Alemany Boulevard',
 'Broadway (San Francisco)|Broadway',
 'The Castro, San Francisco|Castro Street',
 'Cesar Chavez Street',
 'Divisadero Street',
 'Don Chee Way',
 'Embarcadero (San Fran

In [25]:
with open(f"../output/street_type_designations.txt", 'r+', encoding="utf-8") as filepath:
    doc = filepath.read()
street_types = doc.split('\n')
street_types

['Highway',
 'Autobahn',
 'Auto-estrada',
 'Autoroute',
 'Autostrada',
 'Autostrasse',
 'Bypass',
 'Expressway',
 'Freeway',
 'Motorway',
 'Pike',
 'Turnpike',
 'Avenue',
 'Boulevard',
 'Parade',
 'Road',
 'Street',
 'Arcade',
 'Alley',
 'Bay',
 'Branch',
 'Brook',
 'Burg',
 'Byway',
 'Camp',
 'Center',
 'Club',
 'Common',
 'Corner',
 'Course',
 'Dale',
 'Divide',
 'Drive',
 'Estate',
 'Flat',
 'Forge',
 'Fork',
 'Fort',
 'Gardens',
 'Gate',
 'Gateway',
 'Glen',
 'Green',
 'Grove',
 'Harbor',
 'Haven',
 'Heights',
 'Highlands',
 'Hollow',
 'Key',
 'Knoll',
 'Landing',
 'Lane',
 'Light',
 'Loaf',
 'Lock',
 'Lodge',
 'Manor',
 'Meadow',
 'Mews',
 'Mill',
 'Mission',
 'Neck',
 'Orchard',
 'Passage',
 'Path',
 'Pathway',
 'Ranch',
 'Rapid',
 'Rest',
 'Route',
 'Row',
 'Rue',
 'Run',
 'Station',
 'Terrace',
 'Throughway',
 'Trace',
 'Track',
 'Trafficway',
 'Trail',
 'Trailer',
 'Union',
 'Vale',
 'View',
 'Village',
 'Ville',
 'Vista',
 'Walk',
 'Wall',
 'Way',
 'Well',
 'Wynd',
 'Cul-de-s

In [43]:
all_items = []
for section, items in section_dict.items():
    for index, item in enumerate(items):
        #print(item)
        item = re.sub(r'\[|\]', '', item)
        if '<ref' in item:
            #print(item)
            all_items.append(re.search(r'(.*)\<ref', item).group(1))
        elif '|' in item:
            #print(item)
            #all_items.append(re.search(r'\|(.*)', item).group(1))
            for ytem in item.split('|'):
                all_items.append(ytem.strip())
        elif 'and ' in item:
            print(f"### and-split {index}")
            if ',' in item:
                #print(f"comma-split{item.split(',')}")
                item = re.sub(r'and ', ',', item)
                #print(f"### comma-split {index}")
                for ytem in item.split(','):
                    all_items.append(ytem.strip())
            else:
                #print(f"and-split{item.split('and')}")
                #print(f"### and-split {index}")
                for ytem in item.split('and '):
                    all_items.append(ytem.strip())
        elif '& ' in item:
            for ytem in item.split('& '):
                    all_items.append(ytem.strip())
        elif ',' in item:
            #print(f"comma-split{item.split(',')}")
            print(f"### comma-split {index}")
            for ytem in item.split(','):
                all_items.append(ytem.strip())

        else:
            all_items.append(item)


In [44]:
#for index,item in enumerate(all_items): print(index, item)

In [45]:
for index, item in enumerate(all_items):
    if '#' in item:
        #print(item)
        #print([ytem for ytem in item.split('#')])
        #print(all_items[:index])
        #print(all_items[index + 1:])
        all_items = all_items[:index-1] + [ytem for ytem in item.split('#')] + all_items[index + 1:]
    if ',' in item:
        all_items = all_items[:index-1] + [ytem for ytem in item.split(',')] + all_items[index + 1:]
            
all_items

['19th Avenue (San Francisco)',
 '19th Avenue',
 'California Street (San Francisco)',
 'California Street',
 'Fell Street',
 'Geary Boulevard',
 'Grant Avenue',
 'Fulton Street(San Francisco)',
 'Fulton Street',
 'Lincoln Way (San Francisco)',
 'Lincoln Way',
 'Lombard Street (San Francisco)',
 'Lombard Street',
 'Market Street (San Francisco)',
 'Market Street',
 'Park Presidio Boulevard',
 'Portola Drive',
 'Van Ness Avenue (San Francisco)',
 'Van Ness Avenue',
 '24th Street (San Francisco)',
 '24th Street',
 'Columbus Avenue (San Francisco)',
 'Columbus Avenue',
 'Fillmore Street',
 'Kearny Street',
 'Mission Street',
 'Polk Street',
 'Stockton Street (San Francisco)',
 'Stockton Street',
 'Union Street (San Francisco)',
 'Union Street',
 'Third Street (San Francisco)',
 'Third Street',
 '22nd Street (San Francisco)',
 '49-Mile Scenic Drive',
 'Alemany Boulevard',
 'Broadway (San Francisco)',
 'The Castro',
 ' San Francisco',
 'Castro Street',
 'Cesar Chavez Street',
 'Divisadero St

In [46]:
if '#' in 'Jean Parisot de Valette#Jean De Valette Square': print('yes')

yes


In [47]:
for index, item in enumerate(all_items):
    #remove digits
    item = re.sub('\d+$', '', item).strip()
    # remove comments in parens
    item = re.sub(r'\((.*)\)', '', item).strip()
    # check if last word is street type
    # sometimes both words in street types will have to revalidate when creating
    if len(item.split(' ')) > 1:
        last_word = item.split(' ')[-1]
    if last_word in street_types:
        #print(last_word)
        #print(item)
        all_items[index] = item.replace(last_word, '').strip()
    else:
        all_items[index] = item
        
        #print(item)
all_items

['19th',
 '19th',
 'California',
 'California',
 'Fell',
 'Geary',
 'Grant',
 'Fulton',
 'Fulton',
 'Lincoln',
 'Lincoln',
 'Lombard',
 'Lombard',
 'Market',
 'Market',
 'Park Presidio',
 'Portola',
 'Van Ness',
 'Van Ness',
 '24th',
 '24th',
 'Columbus',
 'Columbus',
 'Fillmore',
 'Kearny',
 'Mission',
 'Polk',
 'Stockton',
 'Stockton',
 'Union',
 'Union',
 'Third',
 'Third',
 '22nd',
 '49-Mile Scenic',
 'Alemany',
 'Broadway',
 'The Castro',
 'San Francisco',
 'Castro',
 'Cesar Chavez',
 'Divisadero',
 'Don Chee',
 'Embarcadero',
 'The Embarcadero',
 'Filbert',
 'Filbert',
 'Golden Gate',
 'Great',
 'Haight',
 'Hayes',
 'Junipero Serra',
 'John F. Kennedy',
 'Lombard',
 'Lombard',
 'Montgomery',
 'New Montgomery',
 'Octavia',
 'California State',
 'Skyline',
 'Vermont',
 'Vermont',
 'Howard',
 'Balmy',
 'Belden',
 'Clarion',
 'Jack Kerouac',
 'Macondray',
 'Maiden',
 'Maiden']

In [48]:
dedupe_all_items = []
for item in all_items:
    if item not in dedupe_all_items:
        dedupe_all_items.append(item)
dedupe_all_items

['19th',
 'California',
 'Fell',
 'Geary',
 'Grant',
 'Fulton',
 'Lincoln',
 'Lombard',
 'Market',
 'Park Presidio',
 'Portola',
 'Van Ness',
 '24th',
 'Columbus',
 'Fillmore',
 'Kearny',
 'Mission',
 'Polk',
 'Stockton',
 'Union',
 'Third',
 '22nd',
 '49-Mile Scenic',
 'Alemany',
 'Broadway',
 'The Castro',
 'San Francisco',
 'Castro',
 'Cesar Chavez',
 'Divisadero',
 'Don Chee',
 'Embarcadero',
 'The Embarcadero',
 'Filbert',
 'Golden Gate',
 'Great',
 'Haight',
 'Hayes',
 'Junipero Serra',
 'John F. Kennedy',
 'Montgomery',
 'New Montgomery',
 'Octavia',
 'California State',
 'Skyline',
 'Vermont',
 'Howard',
 'Balmy',
 'Belden',
 'Clarion',
 'Jack Kerouac',
 'Macondray',
 'Maiden']

In [49]:
# remove empty string if caused eg 'Pellier Avenue & Court'
dedupe_all_items = [item for item in dedupe_all_items if item != '']
dedupe_all_items

['19th',
 'California',
 'Fell',
 'Geary',
 'Grant',
 'Fulton',
 'Lincoln',
 'Lombard',
 'Market',
 'Park Presidio',
 'Portola',
 'Van Ness',
 '24th',
 'Columbus',
 'Fillmore',
 'Kearny',
 'Mission',
 'Polk',
 'Stockton',
 'Union',
 'Third',
 '22nd',
 '49-Mile Scenic',
 'Alemany',
 'Broadway',
 'The Castro',
 'San Francisco',
 'Castro',
 'Cesar Chavez',
 'Divisadero',
 'Don Chee',
 'Embarcadero',
 'The Embarcadero',
 'Filbert',
 'Golden Gate',
 'Great',
 'Haight',
 'Hayes',
 'Junipero Serra',
 'John F. Kennedy',
 'Montgomery',
 'New Montgomery',
 'Octavia',
 'California State',
 'Skyline',
 'Vermont',
 'Howard',
 'Balmy',
 'Belden',
 'Clarion',
 'Jack Kerouac',
 'Macondray',
 'Maiden']

In [153]:
limit = 'all'
with open(f"../output/{topic}_{subtopic}_{limit}.txt", 'w+', encoding="utf-8") as fp:
    for item in cities:
        fp.write(item + '\n')

In [ ]:
# use if items all are/begin with list of links [[link]]

items = []
descriptions = []
with open(topic + '_error_log.txt', 'w+', encoding="utf-8") as filepath:
    for index, line in enumerate(lines[77:291]):
        
        if re.match('\*\s*\[\[', line[0:5]):
            #print("%%%Match%%%")
            #print(line)
            try:
                line = re.sub(r'\<ref.*?\</ref>|\<ref.*\/>', '', line)
                
            except:
                topic_error_line = f"{topic} line sub parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            try:
                
                item = re.search(r'\[\[(.*?)\]\]', line).group(1)
                
            except:
                topic_error_line = f"{topic} item parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            try:
                
                description = re.search(r'\]\]\:*(.*)', line).group(1)
                
                
            except:
                topic_error_line = f"{topic} description parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            items.append(item)
            descriptions.append(description)
        else:
            continue
            #print("%%%No%%%")
            #print(line)

In [ ]:
items = [re.sub(r'\:wiktionary\:', '', item) for item in items]
items = [re.sub(r'\|.*', '', item) for item in items]
items

In [ ]:
# use to find info page for each item/link and rank 
item_infos = []
for index, title in enumerate(items):
    with open(topic + '_error_log.txt', 'w+', encoding="utf-8") as filepath:
        
        url = 'https://en.wikipedia.org/w/index.php?title=' + title + '&action=info'

        #print(url)
        response = requests.get(url)
        doc = lxml.html.fromstring(response.content)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        try:
            # info_table = doc.xpath('//*[@class="wikitable mw-page-info"]')[0]
            watchers = doc.xpath('//*[@id="mw-pageinfo-watchers"]')[0][1].text
            
        except:
            topic_error_line = f"watchers parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            watchers = "#error#"

        try:
            redirects = doc.xpath('//*[contains(text(),"redirects")]/../../td')[1].text
            #redirects = doc.xpath('//*[contains(text(),"redirects")]/parent::node()/parent::node()')
            #redirects = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[2]/tr[9]/td')[1].text
            
        except:
            topic_error_line = f"redirects parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            redirects = "#error#"
             
        try:
            views = doc.xpath('//*[@id="mw-pvi-month-count"]/td/div')[0].text
            
        except:
            topic_error_line = f"views parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            views = "#error#"        
        
        try:
            edits = doc.xpath('//*[@id="mw-pageinfo-edits"]/td')[1].text
            #edits = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[4]/tr[5]/td')[1].text
    
        except:
            topic_error_line = f"edits parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            edits = "#error#"
        
        item_infos.append({
            topic: title,
            'watchers': watchers,
            'redirects': redirects,
            'views': views,
            'edits': edits
        })


In [ ]:
# transfer item_infos to json file
import json
with open(topic + '_infos.json', 'w') as filepath:
    json.dump(item_infos, filepath)

In [ ]:
error_items = [item[topic] for item in item_infos if item['views'] == '#error#']

for index, title in enumerate(error_items):
    with open('info_error_log.txt', 'w+', encoding="utf-8") as filepath:
        #print(title)
        del_idx = 0
        for index, item in enumerate(item_infos):
            if item[topic] == title:
                del_idx = index
                #del item_infos[index]
        #print(item)
        
        
        if re.match(r'\W*', title):
            print('%%%')
            search = re.search(r'(\w+)?', title).group(1)
            print(search)
            title = search
        
        print(title)
        
        url = 'https://en.wikipedia.org/w/index.php?title=' + title + '&action=info'

        #print(url)
        response = requests.get(url)
        doc = lxml.html.fromstring(response.content)
        soup = BeautifulSoup(response.content, 'html.parser')
  
        try:
            # info_table = doc.xpath('//*[@class="wikitable mw-page-info"]')[0]
            watchers = doc.xpath('//*[@id="mw-pageinfo-watchers"]')[0][1].text
            
        except:
            topic_error_line = f"watchers parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            watchers = "#error#"

        try:
            redirects = doc.xpath('//*[contains(text(),"redirects")]/../../td')[1].text
            #redirects = doc.xpath('//*[contains(text(),"redirects")]/parent::node()/parent::node()')
            #redirects = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[2]/tr[9]/td')[1].text
            
        except:
            topic_error_line = f"redirects parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            redirects = "#error#"
             
        try:
            views = doc.xpath('//*[@id="mw-pvi-month-count"]/td/div')[0].text
            
        except:
            topic_error_line = f"views parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            views = "#error#"        
        
        try:
            edits = doc.xpath('//*[@id="mw-pageinfo-edits"]/td')[1].text
            #edits = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[4]/tr[5]/td')[1].text
    
        except:
            topic_error_line = f"edits parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            edits = "#error#"
            
        item_infos[del_idx] = {
            topic: title,
            'watchers': watchers,
            'redirects': redirects,
            'views': views,
            'edits': edits
        }

In [ ]:
with open('mineral_infos.json', 'r') as jfile:
    data=jfile.read()

# parse file
item_infos = json.loads(data)

In [ ]:
_item_infos = item_infos
len(_item_infos)

In [ ]:
#item_infos = [{item['views']: int(item['views'].replace(',', ''))} for item in item_infos]
for index, item in enumerate(_item_infos):
    if item['views'] != '#error#':
        if item['watchers'] == 'Fewer than 30 watchers':
            item['watchers'] = '29'
        item['views'] = int(item['views'].replace(',', ''))
        item['watchers'] = int(item['watchers'].replace(',', ''))
        #item['edits'] = int(item['edits'].replace(',', ''))
        
        
        

In [ ]:
by_views = sorted(_item_infos, key=lambda k : k['views'], reverse=True)
for index, item in enumerate(by_views):
    item['view_rank'] = index

In [ ]:
by_edits = sorted(_item_infos, key=lambda k : k['edits'], reverse=True)
for index, item in enumerate(by_edits):
    item['edit_rank'] = index

In [ ]:
by_watchers = sorted(_item_infos, key=lambda k : k['watchers'], reverse=True)
for index, item in enumerate(by_watchers):
    item['watcher_rank'] = index

In [ ]:
for item in _item_infos:
    #item['total_rank'] = (item['view_rank'] + item['edit_rank'] + item['watcher_rank']) / 3
    item['total_rank'] = (item['view_rank'] + item['watcher_rank']) / 2

    

In [ ]:
by_total = sorted(_item_infos, key=lambda k : k['total_rank'], reverse=True)

In [ ]:
dedupe = []
for item in by_total:
    if item[topic] not in dedupe:
        dedupe.append(item[topic])
        

In [ ]:
start = 0
limit = 44
top_items = dedupe[start:limit]

In [ ]:
with open(f"{topic}_{limit}.txt", 'w+', encoding="utf-8") as fp:
    for item in top_items:
        fp.write(item + '\n')

In [ ]:
min_df = pd.DataFrame(_item_infos)
min_df.head()
min_df.sort_values(by=['total_rank'], ascending=False).head(100)

In [ ]:
min_df = pd.DataFrame(_item_infos)
min_df.head()
min_df.sort_values(by=['views'], ascending=False).head(100)

In [ ]:
min_df.sort_values(by=['watchers'], ascending=False).head(100)

In [ ]:
min_df.sort_values(by=['edits'], ascending=False).head(100)

In [ ]:
# parse data

lines = textDATA["parse"]["wikitext"]["*"].split('\n')[3:-10] # [::-1]
this_sec = {}
sec = ''
sub_sec = ''
with open('error_lines.txt', 'w+', encoding="utf-8") as filepath:
    for index, line in enumerate(lines):
        if index == 23:
            print(line)
        if re.match('==\w', line[0:3]):
            sec = line.strip().replace("=", "")
            if len(this_sec) > 0:
                section_dict[secs[secs.index(sec) - 1]] = this_sec
            this_sec = {}
        if type(section_dict[sec]).__name__ == 'list':
            if re.match('\*\[\[', line[0:3]):
                try:
                    poet = re.search(r'\[\[(.*?)\]\]', line).group(1)
                    #print(poet)

                except:
                    print("##########################")
                    print("#### poet parse error ####")
                    print(poet_method, index, line)
                    filepath.write(line + '\n')
                    print(line[-2:])
                    print("##########################")

                date_m = re.search('\((.*)\)', line)

                try:
                    date = date_m.group(0)
                except:
                    print("##########################")
                    print("#### date parse error ####")
                    print(index, line)
                    filepath.write(line + '\n')
                    date = ""
                    print("##########################")
                try:
                    # description = re.search('\)*,*.*', line).group(1)
                    # description = re.search('\),+.*', line).group(1)
                    # description = re.search('\]\s.*\)(.*)', line).group(1)
                    description = re.search('\]\](\)*,*.*)', line).group(1)
                except Exception as e:
                    print("#################################")
                    print("#### description parse error ####")
                    print(e)
                    print(index, line)
                    filepath.write(line + '\n')
                    print("#################################")
                # print(this_sec)
                # print(sub_sec)
                section_dict[sec].append({
                    'poet': poet,
                    'date': date,
                    'description': description
                })
        else:
            #print(sec)
            if re.match('===\w', line[0:4]):
                sub_sec = line.strip().replace("=", "")
                #print(sub_sec)
            if line.strip().replace("=", "") in section_dict[sec]:
                #print(sub_sec)
                this_sec[sub_sec] = []
            if sub_sec != '':
                if re.match('\*\[\[', line[0:3]):
                    try:
                        poet = re.search(r'\[\[(.*?)\]\]', line).group(1)
                        #print(poet)
                    except:
                        print("##########################")
                        print("#### poet parse error ####")
                        print(index, line)
                        filepath.write(line + '\n')
                        print(line[-2:])
                        print("##########################")
                    date_m = re.search('\((.*)\)', line)
                    try:
                        date = date_m.group(0)
                    except Exception as e:
                        print("##########################")
                        print("#### date parse error ####")
                        print(e)
                        print(index, line)
                        filepath.write(line + '\n')
                        date = ""
                        print("##########################")
                    try:
                        # description = re.search('\)*,*.*', line).group(1)
                        # description = re.search('\),+.*', line).group(0)
                        # description = re.search('\]\s.*\)(.*)', line).group(0)
                        description = re.search('\]\](\)*,*.*)', line).group(1)

                    except Exception as e:
                        print("#################################")
                        print("#### description parse error ####")
                        print(e)
                        print(index, line)
                        filepath.write(line + '\n')
                        description = ""
                        print("#################################")

                    this_sec[sub_sec].append({
                        'poet': poet,
                        'date': date,
                        'description': description
                    })
